# Working with Time Zones

## Ambiguous and imaginary times

### Ambiguous times
An *ambiguous* time is when the same "wall time" occurs more than once, such as during a DST to STD transition.

In [ ]:
from datetime import datetime, timedelta

from dateutil import tz

import pytz

import tz_tests
from helper_functions import print_dt_tzinfo

In [ ]:
NYC = tz.gettz('America/New_York')

In [ ]:
dt1 = datetime(2004, 10, 31, 4, 30, tzinfo=tz.UTC)
for i in range(4):
    dt = (dt1 + timedelta(hours=i)).astimezone(NYC)
    ambig_str = 'Ambiguous' if tz.datetime_ambiguous(dt) else 'Unambiguous'
    print(f'{dt} | {dt.tzname()} |  {ambig_str}')

### [PEP 495: Local Time Disambiguation](https://www.python.org/dev/peps/pep-0495/)
- First introduced in Python 3.6
- Introduces the `fold` attribute of `datetime`
- Changes aware datetime comparison around ambiguous times

PEP 495 makes whether you are on the fold side a *property of the `datetime`*:

In [ ]:
print_dt_tzinfo(datetime(2004, 10, 31, 1, 30, tzinfo=NYC))
print('')
print_dt_tzinfo(datetime(2004, 10, 31, 1, 30, fold=1, tzinfo=NYC))

**Note**: `fold=1` represents the *second* instance of an ambiguous `datetime`.

### Imaginary times
*Imaginary* times are wall times that never occur in a given time zone, such as during an STD to DST transition:

In [ ]:
dt1 = datetime(2004, 4, 4, 6, 30, tzinfo=tz.UTC)
for i in range(3):
    dt = (dt1 + timedelta(hours=i)).astimezone(NYC)
    print(f'{dt} | {dt.tzname()} ')

### Handling ambiguous times

Both `dateutil` and `pytz` will automatically give you the right *absolute time* if converting from an absolute time.

In [ ]:
dt1 = datetime(2004, 10, 31, 6, 30, tzinfo=tz.UTC)   # This is in the fold in EST

In [ ]:
# dateutil
dt_du = dt1.astimezone(tz.gettz('America/New_York'))
print(repr(dt_du))
print_dt_tzinfo(dt_du)

In [ ]:
# pytz
dt_pytz = dt1.astimezone(pytz.timezone('America/New_York'))
print(repr(dt_pytz))    # Note that pytz doesn't set the fold attribute
print_dt_tzinfo(dt_pytz)

For backwards compatibility, `dateutil` provides a `tz.enfold` method to add a `fold` attribute if necessary:

In [ ]:
dt = datetime(2004, 10, 31, 1, 30, tzinfo=NYC)
tz.enfold(dt)

```python
Python 2.7.12
Type "help", "copyright", "credits" or "license" for more information.
>>> from datetime import datetime
>>> from dateutil import tz
>>> dt = datetime(2004, 10, 31, 1, 30, tzinfo=tz.gettz('US/Eastern'))
>>> tz.enfold(dt)
_DatetimeWithFold(2004, 10, 31, 1, 30, tzinfo=tzfile('/usr/share/zoneinfo/US/Eastern'))
>>> tz.enfold(dt).tzname()
'EST'
>>> dt.tzname()
'EDT'
```

To detect ambiguous times, `dateutil` provides `tz.datetime_ambiguous`:

In [ ]:
tz.datetime_ambiguous(datetime(2004, 10, 31, 1, 30, tzinfo=NYC))

In [ ]:
tz.datetime_ambiguous(datetime(2004, 10, 31, 1, 30), NYC)

In [ ]:
dt_0 = datetime(2004, 10, 31, 0, 30, tzinfo=NYC)
for i in range(3):
    dt_i = dt_0 + timedelta(hours=i)
    dt_i = tz.enfold(dt_i, tz.datetime_ambiguous(dt_i))
    print(f'{dt_i} (fold={dt_i.fold})')

**Note:** `fold` is ignored when `datetime` is not ambiguous:

In [ ]:
for i in range(3):
    dt_i = tz.enfold(dt_0 + timedelta(hours=i), fold=1)
    print(f'{dt_i} (fold={dt_i.fold})')

### Handling imaginary times

While functions that convert from an *absolute time* to another absolute time (e.g. `astimezone`, `fromtimestamp` and `now`) will never create an imaginary time, it is possible to create imaginary times with functions that manipulate the naïve portion of the date, such as the constructor, arithmetic and `replace`.

In order to determine if you have created an imaginary time after one of these operations, you can use `dateutil`'s `tz.datetime_exists()`:

In [ ]:
dt_0 = datetime(2004, 4, 4, 1, 30, tzinfo=NYC)
for i in range(3):
    dt = dt_0 + timedelta(hours=i)
    print(f'{dt} ({{}})'.format('Exists' if tz.datetime_exists(dt) else 'Imaginary'))

For the most part, if you are creating imaginary times, you want to "skip forward", to what the time *would be* if the transition had not happened: for example, if the datetime you have created is the result of an addition operation. For this, `dateutil` provides the `tz.resolve_imaginary` function:

In [ ]:
dt = datetime(2004, 4, 4, 1, 30, tzinfo=NYC)
dt_imag = dt + timedelta(hours=1)   # 2004-04-04 02:30 is imaginary
print(f"Imaginary: {dt_imag}")
print(f"Resolved:  {tz.resolve_imaginary(dt_imag)}")

This works for imaginary times other than 1 hour as well:

In [ ]:
dt = datetime(1994, 12, 31, 9, tzinfo=tz.gettz('Pacific/Kiritimati'))
print(f'{dt} ({{}})'.format('Exists' if tz.datetime_exists(dt) else 'Imaginary'))

In [ ]:
tz.resolve_imaginary(dt)

All of these functions for handling ambiguous and imaginary times will work for `pytz` time zones as well, though `pytz` has its own way of handling this.

### `pytz`'s approach to handling ambiguous and imaginary times
#### Ambiguous times

`pytz` predates PEP 495, and in many ways its design was intended to solve the ambiguous and imaginary time problem before there was support for doing so in the standard library. Much of its non-standard interface is a consequence of the fact that without eagerly calculating time zone offsets, there is no way to specify which ambiguous wall time your `datetime` represents.

When localizing an ambiguous `datetime`, `pytz` will default to the *second* occurrence (i.e. the `fold=1` state, usually standard time):

In [ ]:
NYC_pytz = pytz.timezone('America/New_York')
dt_pytz = NYC_pytz.localize(datetime(2004, 10, 31, 1, 30))
print_dt_tzinfo(dt_pytz)

To get the *first* occurrence of a given wall time (i.e. the `fold=0` state, usually daylight saving time), pass `is_dst=True` to the `localize` function:

In [ ]:
dt_pytz = NYC_pytz.localize(datetime(2004, 10, 31, 1, 30), is_dst=True)
print_dt_tzinfo(dt_pytz)

If you want to *detect* ambiguous times, pass `is_dst=None`, and `pytz` will raise an `AmbiguousTimeError` if a datetime is ambiguous:

In [ ]:
for hour in (0, 1):
    dt = datetime(2004, 10, 31, hour, 30)
    try:
        NYC_pytz.localize(dt, is_dst=None)
        print(f'{dt} | Unambiguous')
    except pytz.AmbiguousTimeError:
        print(f'{dt} | Ambiguous')

#### Imaginary times
When using `localize` on an imaginary `datetime`, `pytz` will create an imaginary time and assign it an offset based on `is_dst`:

In [ ]:
print(NYC_pytz.localize(datetime(2004, 4, 4, 2, 30), is_dst=True))
print(NYC_pytz.localize(datetime(2004, 4, 4, 2, 30), is_dst=False))

The default for `is_dst` is `False`:

In [ ]:
print(NYC_pytz.localize(datetime(2004, 4, 4, 2, 30)))

Again, setting `is_dst=None` will cause `pytz` to throw an error, this time `NonExistentTimeError`:

In [ ]:
dt_0 = datetime(2004, 4, 4, 1, 30)
for i in range(3):
    try:
        dt = NYC_pytz.localize(dt_0 + timedelta(hours=i), is_dst=None)
        exist_str = 'Exists'
    except pytz.NonExistentTimeError:
        exist_str = 'Imaginary'

    print(f'{dt} ({exist_str})')

### Exercise: Build a `pytz`-style exception-based localizer with `dateutil`

While `dateutil`'s interface for handling imaginary and ambiguous times is compatible with `pytz` zones, there is no built-in mechanism to throw *exceptions* when imaginary or ambiguous times are created. As an exercise, try to build a `localize` function that throws an exception using `dateutil.tz`'s functions for handling ambiguous and imaginary datetimes.

In [ ]:
from tz_answers import AmbiguousTimeError, NonExistentTimeError

def localize(dt, tzi, is_dst=False):
    pass

In [ ]:
### Uncomment this to test
# tz_tests.test_localize(localize)